Modification of the code that Karthik gave to me.

In [1]:
import numpy as np
import pathlib
import pyvisa
import time
import matplotlib.pyplot as plt
import math
from datetime import datetime
TODAY_STR = datetime.now().strftime("%Y%m%d")
print(f"\x1b[1;3;4;96mDate of running the code:\x1b[0m \x1b[1;3;4;92m{TODAY_STR}\x1b[0m")
print(f"Time of running the code: {datetime.now().strftime("%H:%M:%S")}")

Date of running the code: 20241018
Time of running the code: 09:55:39


In [2]:
# for book keeping
CHIP_ID = f"Freespace-{TODAY_STR}"
FEATURE_ID = "new_GRIN_Groove_with_YSO_crystal_0.005E_-10DbMr"

PM_RANGE = 10

# FEATURE_ID = 'col2_row_ring_top_weirdcoupling'
print(f"{CHIP_ID} {FEATURE_ID} {PM_RANGE}")

Freespace-20241018 new_GRIN_Groove_with_YSO_crystal_0.005E_-10DbMr 10


In [3]:
# for book keeping
grating_file_name = pathlib.Path(f"{CHIP_ID}_{FEATURE_ID}.npz")
if grating_file_name.is_file():
    print("Found existing file. Plotting...")
    grating_input_data = np.load(grating_file_name)
    data_arr_cut = grating_input_data["data_arr_cut"]
    wav_arr = grating_input_data["wav_arr"]
    del grating_input_data
    plt.close()
    plt.plot(wav_arr, data_arr_cut)
    plt.grid()
    plt.xlabel("wavelength (nm)")
    plt.ylabel("voltage (V)")
    plt.title(f"{CHIP_ID}_{FEATURE_ID} transmission")
    plt.annotate(
        f"Power meter range: {PM_RANGE} dBm",
        (0.15, 0.75),
        xycoords="axes fraction",
        alpha=0.6,
        fontsize=12,
        fontfamily="monospace",
    )
    plt.show()
else:
    print("\x1b[0;93mNo existing file found. Proceeding...\x1b[0m")

No existing file found. Proceeding...


In [4]:
# Always run this: device communication
rm = pyvisa.ResourceManager()
rm.list_resources()

('TCPIP::169.254.112.67::INSTR',
 'TCPIP::169.254.112.67::INSTR',
 'TCPIP::169.254.112.67::INSTR',
 'TCPIP::169.254.112.67::INSTR')

In [5]:
# # oscilloscope connection
# Open the connection to the oscilloscope using its IP address
oscilloscope = rm.open_resource('TCPIP::169.254.112.67::INSTR', open_timeout=5000) # For the RIGOL DHO1204 in A*STAR

# Send a command to verify connection (e.g., identify the instrument)
# Sanity check for successful communication with oscilloscope
response = oscilloscope.query('*IDN?')
print(response)

RIGOL TECHNOLOGIES,DHO1204,HDO1B261900518,00.02.12



PLEASE READ THE FOLLOWING CAREFULLY
1. Turn off roll on the oscilloscope (top of the screen -> press (A), which is to the right of (H) -> Set Roll from Auto to Off)
2. Set MemDepth to at least 1M (default is 10k)
3. Set the time base to something on the order of seconds by adjusting the dial
4. For CH1 (trigger channel), set the voltage (vertical) axis to the order of 4V or so

To reset just press auto on the top RHS of the console

In [6]:
# # Laser connection (laser is TSL)
# is_connect_success = False 
# while not is_connect_success:
#     try:
#         TSL = rm.open_resource("TCPIP::169.254.82.30::5000::SOCKET", read_termination="\r")
#         print(TSL.query("*IDN?"))
#         print("\x1b[0;92mTSL Connection established.\x1b[0m")
#         is_connect_success = True
#     except pyvisa.VisaIOError:
#         print("Retrying...")
#         time.sleep(0.5)

# if TSL.query(":POW:STAT?") == "+0":
#     TSL.write(":POW:STAT 1")
#     time.sleep(300)
#     print("Laser was off. Turning on laser.")

In [7]:
## Some code to start and stop the capturing programmatically

# Function to stop capturing
def stop_running():
    oscilloscope.write(':STOP')
    print("Oscilloscope stopped running and capturing.")

# Function to set up and enable triggering
def setup_trigger(trigger_level, trigger_source='CHAN1'):
    # Set the trigger source (e.g., Channel 1)
    oscilloscope.write(f':TRIGger:EDGE:SOURce {trigger_source}')
    
    # Set the trigger level (e.g., 0.5V)
    oscilloscope.write(f':TRIGger:EDGE:LEVel {trigger_level}')
    
    # Set the trigger type (e.g., edge trigger on a rising edge)
    oscilloscope.write(':TRIGger:EDGE:SLOPe POSitive')
    
    print(f"Trigger set to {trigger_level}V on {trigger_source}.")

# # Function to force a trigger (useful for testing)
# def force_trigger():
#     oscilloscope.write(':TFORce')
#     print("Manual trigger executed.")

def wait_for_first_trigger():
    """
    Waits for the first trigger event to occur. Once detected, it returns True to proceed.
    """
    oscilloscope.write(':RUN') # Need oscilloscope to start running before it can capture anything
    # The trigger will automatically start capturing once the trigger is detected

    print("Waiting for the first trigger...")
    while True:
        trigger_status = oscilloscope.query(':TRIGger:STATus?').strip()  # Possible values: TD, WAIT, RUN, AUTO, STOP
        
        if trigger_status == 'TD':
            print("First trigger detected. Starting capture.")
            return True
            
        # except pyvisa.errors.VisaIOError:
        #     print("Error: Failed to read trigger status. Retrying...")
        #     continue

def monitor_after_first_trigger(timeout_in_seconds):
    """
    Monitors the oscilloscope after the first trigger. If no events occur within the timeout period (default: 800us),
    it stops capturing.
    """
    
    # # Convert timeout from microseconds to seconds
    # timeout_s = timeout_us / 1e6

    # Initialize the start time outside the loop
    start_time = time.time() # time.time counts the seconds since the epoch
    print(f"Start time: {start_time}")

    while True:
        # Query the oscilloscope for its trigger status
        trigger_status = oscilloscope.query(':TRIGger:STATus?').strip()  # Possible values: TD, WAIT, RUN, AUTO, STOP
        
        # Check if the oscilloscope status indicates a trigger event (TD) occurred
        if trigger_status == 'TD':
            # Reset the timer if a trigger event occurred
            start_time = time.time()
            print("Triggered")
    
        # except pyvisa.errors.VisaIOError:
        #     print("Error: Failed to read trigger status. Retrying...")
        #     continue
        
        # Check if the timeout period has passed with no trigger event
        elif (time.time() - start_time) >= timeout_in_seconds:
            stop_running()
            print("No triggers detected within the timeout period. Capture stopped.")
            break

# Set up the trigger
setup_trigger(trigger_level=0.7)  # Unit of trigger_level is V
# Wait for the first trigger to occur before starting capture

if wait_for_first_trigger():
    # Monitor for further triggers and stop if no trigger occurs for 800us
    monitor_after_first_trigger(timeout_in_seconds=1)

Trigger set to 0.7V on CHAN1.
Waiting for the first trigger...


KeyboardInterrupt: 

In [8]:
# # more oscilloscope code

# def getData(DSO, channel=2):
#     DSO.write(f":WAV:SOUR CHAN{int(channel)}")
#     DSO.write(":WAV:POINTS 5500000")
#     sRange = DSO.query("WAV:PRE?")
#     lRange = sRange.split(",")
#     # <format 16-bit NR1>, <type 16-bit NR1>, <points 32-bit NR1>, <count 32-bit NR1>,
#     # <xincrement 64-bit floating point NR3>, <xorigin 64-bit floating point NR3>,
#     # <xreference 32-bit NR1>,
#     # <yincrement 32-bit floating point NR3>, <yorigin 32-bit floating point NR3>,
#     # <yreference 32-bit NR1>
#     # iStep = 256 if int(lRange[0])==0 else 65536
#     # pts = int(lRange[2])
#     dt = float(lRange[4])
#     gain = float(lRange[7])
#     y_offs = float(lRange[8])
#     y_offs_byte = int(lRange[9])
#     # get data as i16, convert to numpy array
#     DSO.write("WAV:DATA?")
#     sHead0 = DSO.read_bytes(count=10)
#     # strip header to find # of points
#     i0 = sHead0.find(b"#")
#     nDig = int(chr(sHead0[i0 + 1]))  # [i0+1:i0+2])
#     nByte = int((sHead0[i0 + 2 : i0 + 2 + nDig]))
#     # read data, including final line feed
#     sData = DSO.read_bytes(count=1 + nByte)
#     # get data to numpy array
#     vData = np.frombuffer(sData[:-1], dtype="uint8")
#     data_arr = gain * (vData) + y_offs - gain * (y_offs_byte)
#     time_arr = np.linspace(0, data_arr.size, data_arr.size) * dt
#     # time.sleep(5)

#     return data_arr, time_arr

In [9]:
# Sanity check for trigger
# TSL.query("TRIG:OUTP:STEP?")

In [10]:
# WAV_START = 1530 # start wavelength
# WAV_END = 1640 # stop wavelength

In [11]:
# # arms the trigger

# TSL.write("WAV:SWE 0")
# time.sleep(0.5)
# TSL.write(F"WAV:SWE:STAR {WAV_START}E-9")
# time.sleep(0.1)
# TSL.write(F"WAV:SWE:STOP {WAV_END}E-9")
# time.sleep(0.1)
# TSL.write("WAV:SWE 1")
# time.sleep(0.1)

In [12]:
# time_range = 10 # sleep time (next cell)
# time_pos = 4

# # # oscilloscope code
# # DSO.write(f":TIM:RANG {time_range}")
# # time.sleep(0.1)
# # DSO.write(f":TIM:POS {time_pos}")
# # time.sleep(0.1)
# # DSO.write("SING")
# # time.sleep(0.1)

In [13]:
# # runs the sweep
# time.sleep(2)
# TSL.write(":TRIG:INP:SOFT")
# time.sleep(time_range*1.1)

In [14]:
# data_arr, time_arr = getData(DSO, channel=1)
# time.sleep(0.1)
# clock_arr, time_arr = getData(DSO, channel=2)
# time.sleep(0.1)

In [15]:
# plt.plot(time_arr, data_arr)
# plt.show()

In [16]:
# plt.plot(time_arr, clock_arr)
# plt.show()

In [17]:
# # start_idx = np.where(clock_arr > 0.5)[0][0]
# end_idx = np.where(clock_arr > 0.5)[0][-1]

In [18]:
# time_arr_cut = time_arr[start_idx:end_idx]
# data_arr_cut = data_arr[start_idx:end_idx]
# clock_arr_cut = clock_arr[start_idx:end_idx]

In [19]:
# wav_arr = np.linspace(WAV_START, WAV_END, time_arr_cut.size)

In [20]:
# chip_name = f"{CHIP_ID} {FEATURE_ID}"

In [21]:
# plt.close()
# plt.figure(figsize=(16, 10))
# plt.plot(wav_arr, data_arr_cut * 10 ** (PM_RANGE / 10) * 1e-3)
# plt.grid()
# plt.xlabel("wavelength (nm)")
# plt.ylabel("power (W)")
# plt.ylim(0, None)
# # plt.title(f'ESO reflection 1500-1600nm, faster sweep speed')
# plt.title(f"{chip_name} transmission")
# plt.annotate(
#     f"Power meter range: {PM_RANGE} dBm",
#     (0.15, 0.75),
#     xycoords="axes fraction",
#     alpha=0.6,
#     fontsize=12,
#     fontfamily="monospace",
# )
# plt.show()

In [22]:
# plt.close()
# plt.figure(figsize=(16, 10))
# plt.plot(wav_arr, data_arr_cut * 10 ** (PM_RANGE / 10) * 1e-3)
# plt.grid()
# plt.xlabel("wavelength (nm)")
# plt.ylabel("power (W)")
# # plt.title(f'ESO reflection 1500-1600nm, faster sweep speed')
# plt.title(f"{chip_name} transmission")
# plt.annotate(
#     f"Power meter range: {PM_RANGE} dBm",
#     (0.15, 0.75),
#     xycoords="axes fraction",
#     alpha=0.6,
#     fontsize=12,
#     fontfamily="monospace",
# )
# plt.show()

In [23]:
# plt.close()
# plt.figure(figsize=(16, 10))
# watt = list(data_arr_cut * 10 ** (PM_RANGE / 10) * 1e-3)
# dBm = list(map(lambda x: math.log10(x) * 10 + 30, watt))
# plt.plot(wav_arr, dBm)
# plt.grid()
# plt.xlabel("wavelength (nm)")
# plt.ylabel("power (dBm)")
# # plt.title(f'ESO reflection 1500-1600nm, faster sweep speed')
# plt.title(f"{chip_name} transmission")
# plt.annotate(
#     f"Power meter range: {PM_RANGE} dBm",
#     (0.15, 0.75),
#     xycoords="axes fraction",
#     alpha=0.6,
#     fontsize=12,
#     fontfamily="monospace",
# )
# plt.show()

In [24]:
# plt.close()
# plt.plot(wav_arr, data_arr_cut)
# plt.grid()
# plt.xlabel("wavelength (nm)")
# plt.ylabel("voltage (V)")
# plt.title(f"{chip_name} transmission")
# plt.annotate(
#     f"Power meter range: {PM_RANGE} dBm",
#     (0.15, 0.75),
#     xycoords="axes fraction",
#     alpha=0.6,
#     fontsize=12,
#     fontfamily="monospace",
# )
# plt.gcf().set_dpi(200)
# plt.show()

In [25]:
# np.savez(
#     f"{CHIP_ID}_{FEATURE_ID}.npz",
#     time_arr=time_arr,
#     data_arr=data_arr,
#     clock_arr=clock_arr,
#     wav_arr=wav_arr,
#     time_arr_cut=time_arr_cut,
#     data_arr_cut=data_arr_cut,
#     chip_name=chip_name,
#     PM_RANGE=PM_RANGE,
# )

In [26]:
# TSL.write("WAV:SWE 0")

In [27]:
# TSL.write("WAV 1520nm")

In [28]:
# # if TSL.query(":POW:STAT?") == "+1":
# #     TSL.write(":POW:STAT 0") # Turn off laser
# TSL.close()
# DSO.close()
# del TSL
# del DSO